# run deepforest training

In [1]:
from deepforest import deepforest
from deepforest import get_data
import tensorflow as tf
import datetime

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [4]:
data_type = 'NAIP2018'
annotation_type = 'Hand'
area = 'Nubian_square'
notes = 'Robin'

site_name = '{0}_{1}_{2}_{3}'.format(data_type, annotation_type, area, notes)
# site_name = 'Amsterdam_training'

image_type = ".tif"

score_threshold = 0.3

In [5]:
test_model = deepforest.deepforest()
test_model.use_release()

Reading config file: /home/daffolap-830/anaconda3/envs/deepforest/lib/python3.6/site-packages/deepforest/data/deepforest_config.yml
A blank deepforest object created. To perform prediction, either train or load an existing model.
Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/v0.2.1 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/v0.2.1
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, 

In [6]:
# Example run with short training
test_model.config["epochs"] = 20
test_model.config["save-snapshot"] = False
test_model.config["steps"] = 5
test_model.config["batch_size"] = 1
test_model.config['max_queue_size'] = 5
test_model.config['image-min-side'] = 400
test_model.config['save-snapshot'] = True
test_model.config['save_path'] = "../../data/snapshots/" + site_name 
test_model.config['snapshot_path'] = "../../data/snapshots/" + site_name


import os
newpath = "../../data/snapshots/" + site_name 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [7]:
test_model.config

{'batch_size': 4,
 'weights': '/home/daffolap-830/anaconda3/envs/deepforest/lib/python3.6/site-packages/deepforest/data/NEON.h5',
 'backbone': 'resnet50',
 'image-min-side': 400,
 'multi-gpu': 1,
 'epochs': 20,
 'validation_annotations': 'None',
 'freeze_layers': 0,
 'freeze_resnet': False,
 'score_threshold': 0.05,
 'multiprocessing': False,
 'workers': 1,
 'max_queue_size': 5,
 'random_transform': False,
 'save-snapshot': True,
 'save_path': '../../data/snapshots/NAIP2018_Hand_Nubian_square_Robin',
 'snapshot_path': '../../data/snapshots/NAIP2018_Hand_Nubian_square_Robin',
 'steps': 5}

In [14]:
import glob
#Path to sample file
annotations_file_path = '../../data/training_data/' + site_name + '/training_data/*.csv'
# annotations_file_path = '../../data/training_data/' + site_name + '/*.csv'
print(annotations_file_path)
annotations_file = glob.glob(annotations_file_path)[1]
# annotations_file = glob.glob(annotations_file_path)[0]
annotations_file

../../data/training_data/NAIP2018_Hand_Nubian_square_Robin/training_data/*.csv


'../../data/training_data/NAIP2018_Hand_Nubian_square_Robin/training_data/NAIP2018_Hand_Nubian_square_Robin_annotations.csv'

In [15]:
test_model.train(annotations = annotations_file, input_type = "fit_generator")
 
#save trained model
a = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")


lu[0][0]                 
__________________________________________________________________________________________________
bn4f_branch2a (BatchNormalizati (None, None, None, 2 1024        res4f_branch2a[0][0]             
__________________________________________________________________________________________________
res4f_branch2a_relu (Activation (None, None, None, 2 0           bn4f_branch2a[0][0]              
__________________________________________________________________________________________________
padding4f_branch2b (ZeroPadding (None, None, None, 2 0           res4f_branch2a_relu[0][0]        
__________________________________________________________________________________________________
res4f_branch2b (Conv2D)         (None, None, None, 2 589824      padding4f_branch2b[0][0]         
__________________________________________________________________________________________________
bn4f_branch2b (BatchNormalizati (None, None, None, 2 1024        res4f_branch2b[0][

In [10]:
path = '../../data/snapshots/' + site_name + "_" + a + '.h5'
print(path)
test_model.model.save('../../data/snapshots/Amsterdam_training/resnet50_csv_20.h5')

NameError: name 'a' is not defined

In [16]:
mAP = test_model.evaluate_generator(annotations=annotations_file)
print("Mean Average Precision is: {:.3f}".format(mAP))

Running network: N/A% (0 of 38) |        | Elapsed Time: 0:00:00 ETA:  --:--:--There are 1 unique labels: ['tree'] 
Running network: 100% (38 of 38) |#######| Elapsed Time: 0:00:18 Time:  0:00:18
Parsing annotations: 100% (38 of 38) |###| Elapsed Time: 0:00:00 Time:  0:00:00
4114 instances of class tree with average precision: 0.0015
mAP using the weighted average of precisions among classes: 0.0015
mAP: 0.0015
Mean Average Precision is: 0.001


In [12]:
# x = os.listdir('../../data/Amsterdam_chunks/')
# with open('data.txt', 'w+') as txt_file:
#     for name in x:
#         print(name, file=txt_file)
import cv2

path = '../../data/training_data/Amsterdam_training/009_Amsterdam_test.tif'
path = r'/home/daffolap-830/Desktop/green_city_watch_project/TreeTect/data/training_data/Amsterdam_training/009_Amsterdam_test.tif'

img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
print(img)



None
